In [ ]:
from datasets import load_dataset
print('Chargement du ds')
ds = load_dataset("parquet", data_files="data/mmimdb_merged.parquet", split="train[:100%]")
print(ds[0])

Aplanissement du ds 

In [ ]:
def resize(img):
    max_size = (224, 224)  # taille max (width, height)
    img.thumbnail(max_size, Image.Resampling.LANCZOS)
    return img

In [ ]:
import re
def extract_plot(text: str) -> str:
    PATTERN = re.compile(r'(?is)\bplot\s*:\s*(.*?)(?=\bnote\s+that\b|answer\s*:|$)')
    if text is None:
        return None
    m = PATTERN.search(text)
    if not m:
        return ""  # ou text.strip() si tu préfères conserver l'original quand Pas de "Plot:"
    plot = m.group(1).strip()
    # petites normalisations
    plot = re.sub(r'\s+\n', '\n', plot)
    plot = re.sub(r'\s{2,}', ' ', plot)
    return plot
    

In [ ]:
#Creation du one-hot 
def data_loader(ds_flat):
    LABELS = [
        "drama", "comedy", "romance", "thriller", "crime", "action", "adventure", "horror",
        "documentary", "mystery", "sci-fi", "fantasy", "family", "biography", "war", "history",
        "music", "animation", "musical", "western", "sport", "short", "film-noir"
    ]
    label2id = {lab: i for i, lab in enumerate(LABELS)}

    ALIASES = {
        "documentry": "documentary",
        "science fiction": "sci-fi",
        "sci fi": "sci-fi",
        "film noir": "film-noir",
        "westerns": "western",
        "sports": "sport",
    }

    def norm_token(t: str) -> str:
        t = t.lower()
        t = re.sub(r"[\s\-]+", " ", t).strip()
        t = ALIASES.get(t, t)
        t = t.replace("sci fi", "sci-fi")
        return t

    def parse_answer(ans: str):
        if ans is None:
            return set()
        toks = re.split(r"[;,]", ans)
        toks = [norm_token(x) for x in toks if x.strip()]
        return set(t for t in toks if t in label2id)

    def to_multi_hot_float(labels_set):
        vec = [0.0] * len(LABELS)
        for lab in labels_set:
            vec[label2id[lab]] = 1.0
        return vec

    def mapper(batch):
        answers = batch["answer"]
        return {"labels": [to_multi_hot_float(parse_answer(a)) for a in answers]}

    new_features = ds_flat.features.copy()
    new_features["labels"] = Sequence(feature=Value("float32"), length=len(LABELS))

    ds_flat = ds_flat.map(mapper, batched=True, features=new_features)
    print(type(ds_flat[0]["labels"][0]), ds_flat.features["labels"])

    return ds_flat


In [ ]:
import re

LABELS = [
    "drama", "comedy", "romance", "thriller", "crime", "action", "adventure", "horror",
    "documentary", "mystery", "sci-fi", "fantasy", "family", "biography", "war", "history",
    "music", "animation", "musical", "western", "sport", "short", "film-noir"
]
label2id = {lab: i for i, lab in enumerate(LABELS)}

ALIASES = {
    "documentry": "documentary",
    "science fiction": "sci-fi",
    "sci fi": "sci-fi",
    "film noir": "film-noir",
    "westerns": "western",
    "sports": "sport",
}

def norm_token(t: str) -> str:
    t = t.lower()
    t = re.sub(r"[\s\-]+", " ", t).strip()
    t = ALIASES.get(t, t)
    t = t.replace("sci fi", "sci-fi")
    return t

def parse_answer(ans: str):
    if ans is None:
        return set()
    toks = re.split(r"[;,]", ans)
    toks = [norm_token(x) for x in toks if x.strip()]
    return set(t for t in toks if t in label2id)

def to_multi_hot_float(labels_set):
    vec = [0.0] * len(LABELS)
    for lab in labels_set:
        vec[label2id[lab]] = 1.0
    return vec

def data_loader(answer: str):
    labels_set = parse_answer(answer)
    return to_multi_hot_float(labels_set)


In [ ]:
from PIL import Image

def general_dataset_transform(example):
    import json
    Image.MAX_IMAGE_PIXELS = 200_000_000
    msgs = example.get("messages", []) or []
    user_msgs = [m.get("content", "") for m in msgs if m.get("role") == "user"]
    assistant_msgs = [m.get("content", "") for m in msgs if m.get("role") == "assistant"]
    
    image = resize(example.get("images")[0])
    image = pad_to_224_width(image)
    prompt = "\n\n".join([u for u in user_msgs if isinstance(u, str)]) if user_msgs else ""
    plot = extract_plot(prompt)
    answer = assistant_msgs[-1] if assistant_msgs else ""
    
    labels_vec = data_loader(answer)  

    return {
        "plot": plot,
        "answer": answer,
        "image": image,
        "labels": labels_vec, 
    }
    
ds_flat = ds.map(general_dataset_transform, remove_columns=ds.column_names)

In [ ]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["plot"],
            padding="max_length",
            truncation=True)

print('step1')
Dataset_tokenized_images = ds_flat.map(tokenize_function, batched=True)
Dataset_tokenized_images = Dataset_tokenized_images.map(pad_to_224_width)

In [ ]:
print(ds_flat[0]["answer"])
print(ds_flat[0]["labels"])
print(len(ds_flat[0]["labels"]))
print(ds_flat[0]["image"])
print(ds_flat[0]["plot"])

In [ ]:
Dataset_tokenized_images.save_to_disk("data/Dataset_tokenized_images")